Met dit notebook kan de output van D-HYDRO ROI simulaties worden omgezet naar de bestanden die nodig zijn voor het LDO.

Dit notebook heeft 2 opties:
1. output genereren voor één losse simulatie. In dit geval wordt de LDO data in het output mapje geplaatst.
2. output genereren voor meerdere simulaties. Hiervoor moeten de simulaties in één map staan. De LDO output komt in één map te staan die de gebruiker kan opgeven in het notebook.


# Imports and general input

1. Verander **git_dir** in de map op jouw computer met de code. <br>
2. Run vervolgens de cell door met de **Run** knop bovenin het scherm of door **ctrl+enter** te drukken. <br>

In [2]:
import os
import sys
import shutil
import pandas as pd
%load_ext autoreload
%autoreload 2
# insert the directory where you extracted the zip file here:
git_dir = r"C:\Work\Projects\HL-P24050_ROI\05_Analysis\P1414"

sys.path.append(os.path.join(git_dir, 'Code'))
sys.path.append(os.path.join(git_dir, 'HydroLogic_Inundation_toolbox'))
sys.path.append(os.path.join(git_dir, 'HydroLogic_Inundation_toolbox/Readers'))
from notebooks.LDO_export_scripts.ldo_metadata import MetadataGenerator
from notebooks.LDO_export_scripts.dhydro2tif import DhydroToTif
#================== File locations within model directory ==================#

file_locations = {
    'fm_dir': "dflowfm",
    'output_dir': "output",
    'mdu_file': "DFM.mdu",
    'map_file': "map.nc",
    'his_file': "DFM_his.nc",
    'structures_file': "structures.ini",
    'dia_file': "DFM.dia",
}

#===========================================================================#

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Option 1: generate LDO output data for a single model

## 1.1. Generate metadata
Vul de map waarin je model staat in in de variabele **model_dir**. Dit is de map waarin de dflowfm map staat.

In [ ]:
model_dir = r"C:\Work\Projects\HL-P24050_ROI\05_Analysis\SAS_workdir\run_model\test_dambreach\nothing"

metadatagenerator = MetadataGenerator(model_dir, file_locations)
df = metadatagenerator.execute()
metadatagenerator.write(write_as_excel=True)

clean_df_for_notebook =  df.loc[:, ~df.iloc[-1].isna()]
clean_df_for_notebook

## 1.2. Generate TIF files

In [ ]:
t_breach = metadatagenerator.t0
tif_generator = DhydroToTif(model_dir, file_locations, resolution=None)

In [ ]:
tif_generator.create_waterdepth_animation(t_breach) 
tif_generator.create_maximum_waterdepth_tiff()
tif_generator.create_maximum_velocity_tiff()
tif_generator.create_bathymetry_tiff()

# Option 2: Generate LDO output for multiple models
1. Geef met de variabele **directory_with_models** de map op waarin de doorgerekende modellen staan.
2. Geef met de variabele **output_dir** de map op waarin de LDO output moet worden weg geschreven.
Per simulatie wordt een map aangemaakt met tif bestanden. De metadata wordt naar één Excel bestand weg geschreven.

In [4]:
directory_with_models = r"C:\Work\Projects\HL-P24050_ROI\05_Analysis\SAS_workdir\run_model\test_dambreach"
output_dir = r"C:\Work\Projects\HL-P24050_ROI\05_Analysis\SAS_workdir\run_model\TMP_ldo_output"

df = pd.DataFrame()
for model in os.listdir(directory_with_models):
    print("="*30)
    print(f"Working on model: '{model}'")
    model_dir = os.path.join(directory_with_models, model)
    
    try:
        metadatagenerator = MetadataGenerator(model_dir, file_locations, existing_df=df)
        df_out = metadatagenerator.execute()
        df = metadatagenerator.df_
        metadatagenerator.write(
            output_file=os.path.join(output_dir, 'ldo_metadata.xlsx'),
            write_as_excel=True
        )
    except Exception as e:
        print(f"WARNING: Can not create LDO export data for model '{model}', skipping this model. Can not create export data because of error message: {e}")
        continue
    
    tif_output_dir = os.path.join(output_dir, model)
    if os.path.exists(tif_output_dir) == False:
        os.mkdir(tif_output_dir)
        
    tif_generator = DhydroToTif(model_dir, file_locations, resolution=None, silent=True)
    
    t_breach = metadatagenerator.t0
    tif_generator.create_waterdepth_animation(
        t_breach=t_breach,
        output_dir=os.path.join(tif_output_dir, 'waterdepth_animation')
    ) 
    tif_generator.create_maximum_waterdepth_tiff(
        output_file=os.path.join(tif_output_dir, 'maximum-waterdepth.tiff')
    )
    tif_generator.create_maximum_velocity_tiff(
        output_file=os.path.join(tif_output_dir, 'maximum-velocity.tiff')
    )
    tif_generator.create_bathymetry_tiff(
        output_file=os.path.join(tif_output_dir, 'bathymetry.tiff')
    )
    
    print("Output converted succesfully")
    


    


INFO:notebooks.LDO_export_scripts.ldo_metadata:Succesfully wrote the LDO metadata to: C:\Work\Projects\HL-P24050_ROI\05_Analysis\SAS_workdir\run_model\TMP_ldo_output\ldo_metadata.xlsx


Working on model: 'nothing'


INFO:notebooks.LDO_export_scripts.ldo_metadata:Succesfully wrote the LDO metadata to: C:\Work\Projects\HL-P24050_ROI\05_Analysis\SAS_workdir\run_model\TMP_ldo_output\ldo_metadata.xlsx


Output converted succesfully
Working on model: 'original'
Working on model: 'test_ldo_export'


INFO:notebooks.LDO_export_scripts.ldo_metadata:Succesfully wrote the LDO metadata to: C:\Work\Projects\HL-P24050_ROI\05_Analysis\SAS_workdir\run_model\TMP_ldo_output\ldo_metadata.xlsx


...

Output converted succesfully
Working on model: 'upstream_nothing_downstream_xy_but_with_y-500'


INFO:notebooks.LDO_export_scripts.ldo_metadata:Succesfully wrote the LDO metadata to: C:\Work\Projects\HL-P24050_ROI\05_Analysis\SAS_workdir\run_model\TMP_ldo_output\ldo_metadata.xlsx


Output converted succesfully
Working on model: 'upstream_xy_downstream_xy'


INFO:notebooks.LDO_export_scripts.ldo_metadata:Succesfully wrote the LDO metadata to: C:\Work\Projects\HL-P24050_ROI\05_Analysis\SAS_workdir\run_model\TMP_ldo_output\ldo_metadata.xlsx


Output converted succesfully
Working on model: 'upstream_xy_downstream_xy_but_with_upstream_above_downstream_in_txt'
Output converted succesfully
